In [27]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

In [28]:
dataframe = pd.read_csv("transformed_WISDM.csv", header=None)
dataframe.head()
dataframe = dataframe.sample(frac=1).reset_index(drop=True)

In [29]:
dataframe = dataframe.head(2500)

In [30]:
y_train = dataframe.ix[:, 44]

In [31]:
X_train = dataframe.drop(dataframe.columns[[0, 44]], axis=1)

In [32]:
X_train.shape

(2500, 43)

In [33]:
x_numpy = X_train.values

In [34]:
x_numpy

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          3.03993000e-03,   8.46365000e-03,   1.06348000e+01],
       [  1.00000000e-02,   4.65000000e-01,   4.55000000e-01, ...,
          1.06457000e-01,   1.21822000e-01,   1.03333000e+01],
       [  1.75000000e-01,   3.05000000e-01,   3.55000000e-01, ...,
          3.05992000e-01,   2.73435000e-01,   1.11322000e+01],
       ..., 
       [  0.00000000e+00,   8.00000000e-01,   2.00000000e-01, ...,
          4.12984000e-03,   1.11831000e-03,   9.20700000e-02],
       [  2.80000000e-01,   1.60000000e-01,   2.95000000e-01, ...,
          5.85862000e-01,   6.34170000e-01,   1.47466000e+01],
       [  0.00000000e+00,   9.54286000e-01,   4.57143000e-02, ...,
          3.49541000e-03,   4.59256000e-04,   3.52985000e-02]])

In [35]:
from sklearn import preprocessing

In [36]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_numpy)

In [37]:
x_scaled

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          3.01637494e-03,   1.15363625e-02,   1.47269695e-14],
       [  1.00000000e-02,   4.65000000e-01,   4.55000000e-01, ...,
          1.06924181e-01,   1.66839014e-01,   1.43094188e-14],
       [  1.75000000e-01,   3.05000000e-01,   3.55000000e-01, ...,
          3.07406017e-01,   3.74551114e-01,   1.54158243e-14],
       ..., 
       [  0.00000000e+00,   8.00000000e-01,   2.00000000e-01, ...,
          4.11145679e-03,   1.47313573e-03,   1.26225554e-16],
       [  2.80000000e-01,   1.60000000e-01,   2.95000000e-01, ...,
          5.88604059e-01,   8.68763513e-01,   2.04214471e-14],
       [  0.00000000e+00,   9.54286000e-01,   4.57143000e-02, ...,
          3.47401629e-03,   5.70221796e-04,   4.76020721e-17]])

In [38]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train

0        Standing
1         Walking
2         Walking
3         Sitting
4       LyingDown
5        Standing
6         Walking
7         Walking
8         Walking
9         Walking
10        Walking
11       Standing
12        Walking
13        Walking
14         Stairs
15       Standing
16        Sitting
17       Standing
18        Walking
19        Sitting
20        Sitting
21       Standing
22       Standing
23        Sitting
24        Sitting
25        Sitting
26        Walking
27        Sitting
28        Walking
29        Walking
          ...    
2470       Stairs
2471      Walking
2472      Walking
2473      Jogging
2474      Sitting
2475      Sitting
2476      Sitting
2477      Sitting
2478     Standing
2479      Walking
2480     Standing
2481     Standing
2482     Standing
2483      Walking
2484      Walking
2485     Standing
2486      Walking
2487      Jogging
2488      Sitting
2489     Standing
2490     Standing
2491      Walking
2492      Walking
2493      Walking
2494    Ly

In [39]:
dct = {"Jogging" : 1, "LyingDown" : 2, "Sitting" : 3, "Stairs" : 4, "Standing" : 5, "Walking" : 6}
labels = {1:"Jogging", 2:"LyingDown", 3:"Sitting", 4:"Stairs", 5:"Standing", 6:"Walking"}

In [40]:
y_train = y_train.replace(dct)

In [41]:
y_train

0       5
1       6
2       6
3       3
4       2
5       5
6       6
7       6
8       6
9       6
10      6
11      5
12      6
13      6
14      4
15      5
16      3
17      5
18      6
19      3
20      3
21      5
22      5
23      3
24      3
25      3
26      6
27      3
28      6
29      6
       ..
2470    4
2471    6
2472    6
2473    1
2474    3
2475    3
2476    3
2477    3
2478    5
2479    6
2480    5
2481    5
2482    5
2483    6
2484    6
2485    5
2486    6
2487    1
2488    3
2489    5
2490    5
2491    6
2492    6
2493    6
2494    2
2495    2
2496    3
2497    5
2498    6
2499    5
Name: 44, dtype: int64

In [42]:
y_transformed = y_train.values

In [43]:
y_transformed.shape

(2500,)

In [44]:
from sklearn.cross_validation import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_transformed, test_size=0.20, random_state=42)

In [46]:
import sys
import collections
import itertools
from scipy.stats import mode
import numpy as np

try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

class KnnDtw(object):
    """K-nearest neighbor classifier using dynamic time warping
    as the distance measure between pairs of time series arrays
    
    Arguments
    ---------
    n_neighbors : int, optional (default = 5)
        Number of neighbors to use by default for KNN
        
    max_warping_window : int, optional (default = infinity)
        Maximum warping window allowed by the DTW dynamic
        programming function
            
    subsample_step : int, optional (default = 1)
        Step size for the timeseries array. By setting subsample_step = 2,
        the timeseries length will be reduced by 50% because every second
        item is skipped. Implemented by x[:, ::subsample_step]
    """
    
    def __init__(self, n_neighbors=5, max_warping_window=10000, subsample_step=1):
        self.n_neighbors = n_neighbors
        self.max_warping_window = max_warping_window
        self.subsample_step = subsample_step
    
    def fit(self, x, l):
        """Fit the model using x as training data and l as class labels
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
            Training data set for input into KNN classifer
            
        l : array of shape [n_samples]
            Training labels for input into KNN classifier
        """
        
        self.x = x
        self.l = l
        
    def _dtw_distance(self, ts_a, ts_b, d = lambda x,y: abs(x-y)):
        """Returns the DTW similarity distance between two 2-D
        timeseries numpy arrays.

        Arguments
        ---------
        ts_a, ts_b : array of shape [n_samples, n_timepoints]
            Two arrays containing n_samples of timeseries data
            whose DTW distance between each sample of A and B
            will be compared
        
        d : DistanceMetric object (default = abs(x-y))
            the distance measure used for A_i - B_j in the
            DTW dynamic programming function
        
        Returns
        -------
        DTW distance between A and B
        """

        # Create cost matrix via broadcasting with large int
        ts_a, ts_b = np.array(ts_a), np.array(ts_b)
        M, N = len(ts_a), len(ts_b)
        cost = sys.maxint * np.ones((M, N))

        # Initialize the first row and column
        cost[0, 0] = d(ts_a[0], ts_b[0])
        for i in xrange(1, M):
            cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

        for j in xrange(1, N):
            cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

        # Populate rest of cost matrix within window
        for i in xrange(1, M):
            for j in xrange(max(1, i - self.max_warping_window),
                            min(N, i + self.max_warping_window)):
                choices = cost[i - 1, j - 1], cost[i, j-1], cost[i-1, j]
                cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

        # Return DTW distance given window 
        return cost[-1, -1]
    
    def _dist_matrix(self, x, y):
        """Computes the M x N distance matrix between the training
        dataset and testing dataset (y) using the DTW distance measure
        
        Arguments
        ---------
        x : array of shape [n_samples, n_timepoints]
        
        y : array of shape [n_samples, n_timepoints]
        
        Returns
        -------
        Distance matrix between each item of x and y with
            shape [training_n_samples, testing_n_samples]
        """
        
        # Compute the distance matrix        
        dm_count = 0
        
        # Compute condensed distance matrix (upper triangle) of pairwise dtw distances
        # when x and y are the same array
        if(np.array_equal(x, y)):
            x_s = shape(x)
            dm = np.zeros((x_s[0] * (x_s[0] - 1)) // 2, dtype=np.double)
            
            p = ProgressBar(shape(dm)[0])
            
            for i in xrange(0, x_s[0] - 1):
                for j in xrange(i + 1, x_s[0]):
                    dm[dm_count] = self._dtw_distance(x[i, ::self.subsample_step],
                                                      y[j, ::self.subsample_step])
                    
                    dm_count += 1
                    p.animate(dm_count)
            
            # Convert to squareform
            dm = squareform(dm)
            return dm
        
        # Compute full distance matrix of dtw distnces between x and y
        else:
            x_s = np.shape(x)
            y_s = np.shape(y)
            dm = np.zeros((x_s[0], y_s[0])) 
            dm_size = x_s[0]*y_s[0]
            
            p = ProgressBar(dm_size)
        
            for i in xrange(0, x_s[0]):
                for j in xrange(0, y_s[0]):
                    dm[i, j] = self._dtw_distance(x[i, ::self.subsample_step],
                                                  y[j, ::self.subsample_step])
                    # Update progress bar
                    dm_count += 1
                    p.animate(dm_count)
        
            return dm
        
    def predict(self, x):
        """Predict the class labels or probability estimates for 
        the provided data

        Arguments
        ---------
          x : array of shape [n_samples, n_timepoints]
              Array containing the testing data set to be classified
          
        Returns
        -------
          2 arrays representing:
              (1) the predicted class labels 
              (2) the knn label count probability
        """
        
        dm = self._dist_matrix(x, self.x)

        # Identify the k nearest neighbors
        knn_idx = dm.argsort()[:, :self.n_neighbors]

        # Identify k nearest labels
        knn_labels = self.l[knn_idx]
        
        # Model Label
        mode_data = mode(knn_labels, axis=1)
        mode_label = mode_data[0]
        mode_proba = mode_data[1]/self.n_neighbors

        return mode_label.ravel(), mode_proba.ravel()

class ProgressBar:
    """This progress bar was taken from PYMC
    """
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 40
        self.__update_amount(0)
        if have_ipython:
            self.animate = self.animate_ipython
        else:
            self.animate = self.animate_noipython

    def animate_ipython(self, iter):
        print '\r', self,
        sys.stdout.flush()
        self.update_iteration(iter + 1)

    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)

In [47]:
y_transformed[::5].shape

(500,)

In [48]:
from sklearn.decomposition import RandomizedPCA

In [49]:

pca = RandomizedPCA(n_components=6,whiten=True).fit(X_train)

In [50]:
x_pca = pca.transform(X_train)
x_test_pca = pca.transform(X_test)
m = KnnDtw(n_neighbors=1, max_warping_window=10)
m.fit(x_pca, y_train)

In [ ]:
label, proba = m.predict(x_test_pca)

[                  1%                  ]  9779 of 1000000 complete

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
print classification_report(label, y_test,
                            target_names=[l for l in labels.values()])

conf_mat = confusion_matrix(label, y_test)

fig = plt.figure(figsize=(6,6))
width = conf_mat.shape[1]
height = conf_mat.shape[0]

res = plt.imshow(conf_mat, cmap='summer', interpolation='nearest')
for i, row in enumerate(conf_mat):
    for j, c in enumerate(row):
        if c>0:
            plt.text(j-.2, i+.1, c, fontsize=16)
            
cb = fig.colorbar(res)
plt.title('Confusion Matrix')
_ = plt.xticks(range(6), [l for l in labels.values()], rotation=90)
_ = plt.yticks(range(6), [l for l in labels.values()])
plt.show()

             precision    recall  f1-score   support

    Jogging       0.80      0.67      0.73         6
  LyingDown       0.75      0.84      0.79        25
    Sitting       0.77      0.85      0.81        48
     Stairs       0.71      0.83      0.77        12
   Standing       0.90      0.71      0.79        38
    Walking       0.95      0.94      0.95        89

avg / total       0.86      0.86      0.86       218



In [ ]:
conf_mat.shape[1]